In [3]:
import numpy as np
import pandas as pd
import sys
import os
import pickle 

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
import scipy.stats as ss
from sklearn.preprocessing import MultiLabelBinarizer

import random

In [4]:
sys.path.append('./')
from simple_impute import simple_imputer

In [5]:
INTERVENTION = 'vent'
RANDOM = 0
MAX_LEN = 0
SLICE_SIZE = 6
GAP_TIME = 6
PREDICTION_WINDOW = 4
OUTCOME_TYPE = 'all'

In [6]:
CHUNK_KEY = {'ONSET': 0, 'CONTROL': 1, 'ON_INTERVENTION': 2, 'WEAN': 3}

In [7]:
DATAFILE = '~/documents/curated/all_hourly_data.h5'

In [8]:
X = pd.read_hdf(DATAFILE,'vitals_labs')
Y = pd.read_hdf(DATAFILE,'interventions')
static = pd.read_hdf(DATAFILE,'patients')

In [9]:
Y.head()

#Y = Y[[INTERVENTION]]

vent  vaso  adenosine  dobutamine  \
subject_id hadm_id icustay_id hours_in                                      
3          145834  211552     0            1     0          0           0   
                              1            1     1          0           0   
                              2            1     1          0           0   
                              3            1     1          0           0   
                              4            1     1          0           0   

                                        dopamine  epinephrine  isuprel  \
subject_id hadm_id icustay_id hours_in                                   
3          145834  211552     0                0            0        0   
                              1                1            0        0   
                              2                1            0        0   
                              3                0            0        0   
                              4                0            0        0   

                                        milrinone  norepinephrine  \
subject_id hadm_id icustay_id hours_in                              
3          145834  211552     0                 0               0   
                              1                 0               0   
                              2                 0               0   
                              3                 0               0   
                              4                 0               1   

                                        phenylephrine  vasopressin  \
subject_id hadm_id icustay_id hours_in                               
3          145834  211552     0                     0            0   
                              1                     1            0   
                              2                     1            0   
                              3                     1            0   
                              4                     1            0   

                                        colloid_bolus  crystalloid_bolus  \
subject_id hadm_id icustay_id hours_in                                     
3          145834  211552     0                     0                  0   
                              1                     0                  0   
                              2                     0                  0   
                              3                     0                  0   
                              4                     0                  0   

                                        nivdurations  
subject_id hadm_id icustay_id hours_in                
3          145834  211552     0                    0  
                              1                    0  
                              2                    0  
                              3                    0  
                              4                    0

In [10]:
train_ids, test_ids = train_test_split(static.reset_index(), test_size=0.002, 
                                       random_state=RANDOM, stratify=static['mort_hosp'])
split_train_ids, val_ids = train_test_split(train_ids, test_size=0.00125, 
                                            random_state=RANDOM, stratify=train_ids['mort_hosp'])

In [11]:
X_clean = simple_imputer(X,train_ids['subject_id'])


/Users/asjad/anaconda/envs/MIMIC/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/asjad/anaconda/envs/MIMIC/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
X_clean.head()

LEVEL2                                 diastolic blood pressure             \
Aggregation Function                                       mask       mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             1.0  39.666667   
                              1                             1.0  44.125000   
                              2                             1.0  47.333333   
                              3                             1.0  64.500000   
                              4                             1.0  63.000000   

LEVEL2                                                     heart rate          \
Aggregation Function                   time_since_measured       mask    mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0                        0.0        1.0  143.00   
                              1                        0.0        1.0  153.75   
                              2                        0.0        1.0  137.75   
                              3                        0.0        1.0  129.25   
                              4                        0.0        1.0  146.00   

LEVEL2                                                      \
Aggregation Function                   time_since_measured   
subject_id hadm_id icustay_id hours_in                       
3          145834  211552     0                        0.0   
                              1                        0.0   
                              2                        0.0   
                              3                        0.0   
                              4                        0.0   

LEVEL2                                 mean blood pressure              \
Aggregation Function                                  mask        mean   
subject_id hadm_id icustay_id hours_in                                   
3          145834  211552     0                        1.0  101.555567   
                              1                        1.0   57.333337   
                              2                        1.0   63.222217   
                              3                        1.0   84.750000   
                              4                        1.0   77.000000   

LEVEL2                                                     oxygen saturation  \
Aggregation Function                   time_since_measured              mask   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                        0.0               1.0   
                              1                        0.0               0.0   
                              2                        0.0               1.0   
                              3                        0.0               1.0   
                              4                        0.0               1.0   

LEVEL2                                                             \
Aggregation Function                     mean time_since_measured   
subject_id hadm_id icustay_id hours_in                              
3          145834  211552     0         74.00                 0.0   
                              1         74.00                 1.0   
                              2         98.50                 0.0   
                              3         99.75                 0.0   
                              4         98.00                 0.0   

LEVEL2                                 respiratory rate             \
Aggregation Function                               mask       mean   
subject_id hadm_id icustay_id hours_in                               
3          145834  211552     0                     1.0  16.000000   
                              1                     1.0  15.500000   
                              2                     1.0   7.000000   
                      

In [13]:
def minmax(x):# normalize
    mins = x.min()
    maxes = x.max()
    x_std = (x - mins) / (maxes - mins)
    return x_std

In [14]:
def std_time_since_measurement(x):
    idx = pd.IndexSlice
    x = np.where(x==100, 0, x)
    means = x.mean()
    stds = x.std()
    x_std = (x - means)/stds
    return x_std

In [15]:
idx = pd.IndexSlice
X_std = X_clean.copy()
X_std.loc[:,idx[:,'mean']] = X_std.loc[:,idx[:,'mean']].apply(lambda x: minmax(x))
X_std.loc[:,idx[:,'time_since_measured']] = X_std.loc[:,idx[:,'time_since_measured']].apply(lambda x: std_time_since_measurement(x))

In [16]:
X_std.columns = ['_'.join(col) for col in X_std.columns] #drop level of columns and merge with lower level 

X_std.head()

diastolic blood pressure_mask  \
subject_id hadm_id icustay_id hours_in                                  
3          145834  211552     0                                   1.0   
                              1                                   1.0   
                              2                                   1.0   
                              3                                   1.0   
                              4                                   1.0   

                                        diastolic blood pressure_mean  \
subject_id hadm_id icustay_id hours_in                                  
3          145834  211552     0                              0.129207   
                              1                              0.143730   
                              2                              0.154180   
                              3                              0.210098   
                              4                              0.205212   

                                        diastolic blood pressure_time_since_measured  \
subject_id hadm_id icustay_id hours_in                                                 
3          145834  211552     0                                            -0.117221   
                              1                                            -0.117221   
                              2                                            -0.117221   
                              3                                            -0.117221   
                              4                                            -0.117221   

                                        heart rate_mask  heart rate_mean  \
subject_id hadm_id icustay_id hours_in                                     
3          145834  211552     0                     1.0         0.476667   
                              1                     1.0         0.512500   
                              2                     1.0         0.459167   
                              3                     1.0         0.430833   
                              4                     1.0         0.486667   

                                        heart rate_time_since_measured  \
subject_id hadm_id icustay_id hours_in                                   
3          145834  211552     0                              -0.107599   
                              1                              -0.107599   
                              2                              -0.107599   
                              3                              -0.107599   
                              4                              -0.107599   

                                        mean blood pressure_mask  \
subject_id hadm_id icustay_id hours_in                             
3          145834  211552     0                              1.0   
                              1                              1.0   
                              2                              1.0   
                              3                              1.0   
                              4                              1.0   

                                        mean blood pressure_mean  \
subject_id hadm_id icustay_id hours_in                             
3          145834  211552     0                         0.277075   
                              1                         0.137131   
                              2                         0.155767   
                              3                         0.223892   
                              4                         0.199367   

                                        mean blood pressure_time_since_measured  \
subject_id hadm_id icustay_id hours_in                                            
3          145834  211552     0                                       -0.122927   
                              1                                       -0.122927   
                              2                              

In [17]:
del X

In [18]:
def categorize_age(age):
    if age > 10 and age <= 30: 
        cat = 1
    elif age > 30 and age <= 50:
        cat = 2
    elif age > 50 and age <= 70:
        cat = 3
    else: 
        cat = 4
    return cat

def categorize_ethnicity(ethnicity):
    if 'AMERICAN INDIAN' in ethnicity:
        ethnicity = 'AMERICAN INDIAN'
    elif 'ASIAN' in ethnicity:
        ethnicity = 'ASIAN'
    elif 'WHITE' in ethnicity:
        ethnicity = 'WHITE'
    elif 'HISPANIC' in ethnicity:
        ethnicity = 'HISPANIC/LATINO'
    elif 'BLACK' in ethnicity:
        ethnicity = 'BLACK'
    else: 
        ethnicity = 'OTHER'
    return ethnicity

In [19]:
# use gender, first_careunit, age and ethnicity for prediction
static_to_keep = static[['gender', 'age', 'ethnicity', 'first_careunit', 'intime', 'hospital_expire_flag']]
static_to_keep.loc[:, 'intime'] = static_to_keep['intime'].astype('datetime64').apply(lambda x : x.hour)
static_to_keep.loc[:, 'age'] = static_to_keep['age'].apply(categorize_age)
static_to_keep.loc[:, 'ethnicity'] = static_to_keep['ethnicity'].apply(categorize_ethnicity)
static_to_keep = pd.get_dummies(static_to_keep, columns = ['gender', 'age', 'ethnicity', 'first_careunit'])

/Users/asjad/anaconda/envs/MIMIC/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [20]:
static_to_keep.head()

,,,intime,hospital_expire_flag,gender_F,gender_M,age_1,age_2,age_3,age_4,ethnicity_AMERICAN INDIAN,ethnicity_ASIAN,ethnicity_BLACK,ethnicity_HISPANIC/LATINO,ethnicity_OTHER,ethnicity_WHITE,first_careunit_CCU,first_careunit_CSRU,first_careunit_MICU,first_careunit_SICU,first_careunit_TSICU
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,
3,145834,211552,19,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
4,185777,294638,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
6,107064,228232,21,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
9,150750,220597,13,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0
11,194540,229441,6,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0


In [21]:
# merge time series and static data
X_merge = pd.merge(X_std.reset_index(), static_to_keep.reset_index(), on=['subject_id','icustay_id','hadm_id'])
# add absolute time feature
abs_time = (X_merge['intime'] + X_merge['hours_in'])%24
X_merge.insert(4, 'absolute_time', abs_time)
X_merge.drop('intime', axis=1, inplace=True)
X_merge = X_merge.set_index(['subject_id','icustay_id','hadm_id','hours_in'])

In [22]:
del X_std, X_clean

In [23]:
X_merge.columns

Index(['absolute_time', 'diastolic blood pressure_mask',
       'diastolic blood pressure_mean',
       'diastolic blood pressure_time_since_measured', 'heart rate_mask',
       'heart rate_mean', 'heart rate_time_since_measured',
       'mean blood pressure_mask', 'mean blood pressure_mean',
       'mean blood pressure_time_since_measured', 'oxygen saturation_mask',
       'oxygen saturation_mean', 'oxygen saturation_time_since_measured',
       'respiratory rate_mask', 'respiratory rate_mean',
       'respiratory rate_time_since_measured', 'systolic blood pressure_mask',
       'systolic blood pressure_mean',
       'systolic blood pressure_time_since_measured', 'hospital_expire_flag',
       'gender_F', 'gender_M', 'age_1', 'age_2', 'age_3', 'age_4',
       'ethnicity_AMERICAN INDIAN', 'ethnicity_ASIAN', 'ethnicity_BLACK',
       'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_WHITE',
       'first_careunit_CCU', 'first_careunit_CSRU', 'first_careunit_MICU',
       'first

In [24]:
X_merge.shape

(2200954, 37)

In [25]:
mylist = []


temp =0
    
for index, row in Y.iterrows():
    count = 0

    if row['vent'] == 1:
        count = count + 16
    if row['vaso'] == 1:
        count = count + 8
    if row['adenosine'] == 1:
        count = count + 4 
    if row['dobutamine'] == 1:
        count = count + 2
    if row['dopamine'] == 1:
        count = count + 1
    if count == 0:
        temp = temp +1 
    mylist.append(count)


'''
for index, row in Y.iterrows():
    count = 0
    temp = 0 
    if row['vent'] == 1:
        count = 1
    if row['vaso'] == 1:
        count = 2
    if row['vaso'] == 1 and row['vent'] ==1:
        count = 3
    if count > 0:
        mylist.append(count)
    
'''
print ("no action taken #" + str(temp))
print(len(mylist))

no action taken #1629033
2200954


In [26]:
def unique(list1): 
      
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set)) 
    for x in unique_list: 
        print(x) 

unique(mylist) 

0
1
2
3
4
8
9
10
11
12
13
16
17
18
19
20
24
25
26
27
28


In [27]:
#some combination of action(interventions) are never taken and this causes problems later with coach library, so 
#we rearrange the actions so that there are no missing values 


mylist_rearranged = []

for item in mylist:
    if item == 8:
        mylist_rearranged.append(5)
    elif item == 9:
        mylist_rearranged.append(6)
    elif item == 10:
        mylist_rearranged.append(7)
    elif item == 11:
        mylist_rearranged.append(8)
    elif item == 12:
        mylist_rearranged.append(9)
    elif item == 13:
        mylist_rearranged.append(10)
    elif item == 16:
        mylist_rearranged.append(11)
    elif item == 17:
        mylist_rearranged.append(12)
    elif item == 18:
        mylist_rearranged.append(13)
    elif item == 19:
        mylist_rearranged.append(14)
    elif item == 20:
        mylist_rearranged.append(15)
    elif item == 24:
        mylist_rearranged.append(16)
    elif item == 25:
        mylist_rearranged.append(17)
    elif item == 26:
        mylist_rearranged.append(18)
    elif item == 27:
        mylist_rearranged.append(19)
    elif item == 28:
        mylist_rearranged.append(20)
    else:
        mylist_rearranged.append(item)

In [28]:
unique(mylist_rearranged)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [29]:
#X_merge = X_merge.drop(['Action'],axis =1)

X_merge.insert(1, "Action", mylist_rearranged, True) 
X_merge.head()

absolute_time  Action  \
subject_id icustay_id hadm_id hours_in                          
3          211552     145834  0                    19      11   
                              1                    20      17   
                              2                    21      17   
                              3                    22      16   
                              4                    23      16   

                                        diastolic blood pressure_mask  \
subject_id icustay_id hadm_id hours_in                                  
3          211552     145834  0                                   1.0   
                              1                                   1.0   
                              2                                   1.0   
                              3                                   1.0   
                              4                                   1.0   

                                        diastolic blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                                  
3          211552     145834  0                              0.129207   
                              1                              0.143730   
                              2                              0.154180   
                              3                              0.210098   
                              4                              0.205212   

                                        diastolic blood pressure_time_since_measured  \
subject_id icustay_id hadm_id hours_in                                                 
3          211552     145834  0                                            -0.117221   
                              1                                            -0.117221   
                              2                                            -0.117221   
                              3                                            -0.117221   
                              4                                            -0.117221   

                                        heart rate_mask  heart rate_mean  \
subject_id icustay_id hadm_id hours_in                                     
3          211552     145834  0                     1.0         0.476667   
                              1                     1.0         0.512500   
                              2                     1.0         0.459167   
                              3                     1.0         0.430833   
                              4                     1.0         0.486667   

                                        heart rate_time_since_measured  \
subject_id icustay_id hadm_id hours_in                                   
3          211552     145834  0                              -0.107599   
                              1                              -0.107599   
                              2                              -0.107599   
                              3                              -0.107599   
                              4                              -0.107599   

                                        mean blood pressure_mask  \
subject_id icustay_id hadm_id hours_in                             
3          211552     145834  0                              1.0   
                              1                              1.0   
                              2                              1.0   
                              3                              1.0   
                              4                              1.0   

                                        mean blood pressure_mean  ...  \
subject_id icustay_id hadm_id hours_in                            ...   
3          211552     145834  0                         0.277075  ...   
                              1                         0.137131  ...   
                              2                         0.155767  ...   
                              3                  

In [30]:
X_merge.columns

Index(['absolute_time', 'Action', 'diastolic blood pressure_mask',
       'diastolic blood pressure_mean',
       'diastolic blood pressure_time_since_measured', 'heart rate_mask',
       'heart rate_mean', 'heart rate_time_since_measured',
       'mean blood pressure_mask', 'mean blood pressure_mean',
       'mean blood pressure_time_since_measured', 'oxygen saturation_mask',
       'oxygen saturation_mean', 'oxygen saturation_time_since_measured',
       'respiratory rate_mask', 'respiratory rate_mean',
       'respiratory rate_time_since_measured', 'systolic blood pressure_mask',
       'systolic blood pressure_mean',
       'systolic blood pressure_time_since_measured', 'hospital_expire_flag',
       'gender_F', 'gender_M', 'age_1', 'age_2', 'age_3', 'age_4',
       'ethnicity_AMERICAN INDIAN', 'ethnicity_ASIAN', 'ethnicity_BLACK',
       'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_WHITE',
       'first_careunit_CCU', 'first_careunit_CSRU', 'first_careunit_MICU',
   

In [31]:
#remove columns 

X_merge_copy = X_merge.copy()
X_merge_copy = X_merge_copy.drop(['absolute_time', 'diastolic blood pressure_mask', 'diastolic blood pressure_time_since_measured', 'heart rate_mask','heart rate_time_since_measured','mean blood pressure_mask','mean blood pressure_time_since_measured', 'oxygen saturation_mask', 'oxygen saturation_time_since_measured' , 'respiratory rate_mask' ,'respiratory rate_time_since_measured', 'systolic blood pressure_mask','systolic blood pressure_time_since_measured'],axis =1)


In [32]:
X_merge_copy.columns

Index(['Action', 'diastolic blood pressure_mean', 'heart rate_mean',
       'mean blood pressure_mean', 'oxygen saturation_mean',
       'respiratory rate_mean', 'systolic blood pressure_mean',
       'hospital_expire_flag', 'gender_F', 'gender_M', 'age_1', 'age_2',
       'age_3', 'age_4', 'ethnicity_AMERICAN INDIAN', 'ethnicity_ASIAN',
       'ethnicity_BLACK', 'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER',
       'ethnicity_WHITE', 'first_careunit_CCU', 'first_careunit_CSRU',
       'first_careunit_MICU', 'first_careunit_SICU', 'first_careunit_TSICU'],
      dtype='object')

In [33]:
X_merge_copy.fillna(0, inplace=True)

In [34]:

#----------------
# add episode ID
#----------------

mylist_2 =[]

for index, row in X_merge_copy.iterrows():
    mylist_2.append(index[0])


#print(mylist_2)

X_merge_copy.insert(2, "episode_id", mylist_2, True) 


In [35]:
X_merge_copy.head()

Action  diastolic blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                                          
3          211552     145834  0             11                       0.129207   
                              1             17                       0.143730   
                              2             17                       0.154180   
                              3             16                       0.210098   
                              4             16                       0.205212   

                                        episode_id  heart rate_mean  \
subject_id icustay_id hadm_id hours_in                                
3          211552     145834  0                  3         0.476667   
                              1                  3         0.512500   
                              2                  3         0.459167   
                              3                  3         0.430833   
                              4                  3         0.486667   

                                        mean blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                             
3          211552     145834  0                         0.277075   
                              1                         0.137131   
                              2                         0.155767   
                              3                         0.223892   
                              4                         0.199367   

                                        oxygen saturation_mean  \
subject_id icustay_id hadm_id hours_in                           
3          211552     145834  0                         0.7400   
                              1                         0.7400   
                              2                         0.9850   
                              3                         0.9975   
                              4                         0.9800   

                                        respiratory rate_mean  \
subject_id icustay_id hadm_id hours_in                          
3          211552     145834  0                      0.053333   
                              1                      0.051667   
                              2                      0.023333   
                              3                      0.017500   
                              4                      0.045556   

                                        systolic blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                                 
3          211552     145834  0                             0.306002   
                              1                             0.260450   
                              2                             0.291533   
                              3                             0.376206   
                              4                             0.327974   

                                       hospital_expire_flag  gender_F  ...  \
subject_id icustay_id hadm_id hours_in                                 ...   
3          211552     145834  0                           0         0  ...   
                              1                           0         0  ...   
                              2                           0         0  ...   
                              3                           0         0  ...   
                              4                           0         0  ...   

                                        ethnicity_ASIAN  ethnicity_BLACK  \
subject_id icustay_id hadm_id hours_in                                     
3          211552     145834  0                       0                0   
                              1                       0                0   
                              2                       0                0   
                              3                       0                0   
                              4                       0               

In [36]:
# add episode name 

mylist_3 = ['mimic_RL'] * 2200954

X_merge_copy.insert(2, "episode_name", mylist_3, True) 


In [37]:
#add transitions

#X_merge_copy = X_merge_copy.drop(['transition_number'],axis =1)


myindex_list = []
transition_number_list = []

'''
prev_index = 0
for index, row in X_merge_copy.iterrows():
    prev_index = index[0]
    break


counter = 0
for index, row in X_merge_copy.iterrows():
    if prev_index == index[0]:
        transition_number_list.append(counter)
        counter = counter + 1
    else:
        counter = 0
        transition_number_list.append(counter) 
        prev_index = index[0]

'''

for index, row in X_merge_copy.iterrows():
    transition_number_list.append(index[3])

X_merge_copy.insert(4, 'transition_number', transition_number_list, True) 

In [38]:
X_merge_copy.iloc[995:1000]
#X_merge_copy.iloc[10000]
#X_merge_copy.iloc[100000]
#X_merge_copy.iloc[1000000]
#X_merge_copy.iloc[2000000]
#X_merge_copy.iloc[2200000]

Action  diastolic blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                                          
21         217847     109451  141            0                       0.133550   
22         204798     165315  0              0                       0.244571   
                              1             11                       0.211726   
                              2             11                       0.296417   
                              3             11                       0.237785   

                                       episode_name  episode_id  \
subject_id icustay_id hadm_id hours_in                            
21         217847     109451  141          mimic_RL          21   
22         204798     165315  0            mimic_RL          22   
                              1            mimic_RL          22   
                              2            mimic_RL          22   
                              3            mimic_RL          22   

                                        transition_number  heart rate_mean  \
subject_id icustay_id hadm_id hours_in                                       
21         217847     109451  141                     141         0.216667   
22         204798     165315  0                         0         0.343222   
                              1                         1         0.285000   
                              2                         2         0.306667   
                              3                         3         0.306667   

                                        mean blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                             
21         217847     109451  141                       0.164557   
22         204798     165315  0                         0.264416   
                              1                         0.229958   
                              2                         0.319620   
                              3                         0.263713   

                                        oxygen saturation_mean  \
subject_id icustay_id hadm_id hours_in                           
21         217847     109451  141                     0.960000   
22         204798     165315  0                       0.979643   
                              1                       1.000000   
                              2                       0.990000   
                              3                       1.000000   

                                        respiratory rate_mean  \
subject_id icustay_id hadm_id hours_in                          
21         217847     109451  141                    0.060000   
22         204798     165315  0                      0.058000   
                              1                      0.023333   
                              2                      0.023333   
                              3                      0.023333   

                                        systolic blood pressure_mean  ...  \
subject_id icustay_id hadm_id hours_in                                ...   
21         217847     109451  141                           0.398714  ...   
22         204798     165315  0                             0.458199  ...   
                              1                             0.418006  ...   
                              2                             0.524116  ...   
                              3                             0.469453  ...   

                                       ethnicity_ASIAN  ethnicity_BLACK  \
subject_id icustay_id hadm_id hours_in                                    
21         217847     109451  141                    0                0   
22         204798     165315  0                      0                0   
                              1                      0                0   
                              2                      0                0   
                              3                      0               

In [39]:
#remove any dublicate columns
X_merge_copy = X_merge_copy.loc[:,~X_merge_copy.columns.duplicated()]
X_merge_copy.columns
#X_merge_copy.head()

Index(['Action', 'diastolic blood pressure_mean', 'episode_name', 'episode_id',
       'transition_number', 'heart rate_mean', 'mean blood pressure_mean',
       'oxygen saturation_mean', 'respiratory rate_mean',
       'systolic blood pressure_mean', 'hospital_expire_flag', 'gender_F',
       'gender_M', 'age_1', 'age_2', 'age_3', 'age_4',
       'ethnicity_AMERICAN INDIAN', 'ethnicity_ASIAN', 'ethnicity_BLACK',
       'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_WHITE',
       'first_careunit_CCU', 'first_careunit_CSRU', 'first_careunit_MICU',
       'first_careunit_SICU', 'first_careunit_TSICU'],
      dtype='object')

In [40]:
#reorder columns 

X_merge_copy2 = X_merge_copy.filter(['Action', 'episode_id', 'episode_name', 'hospital_expire_flag',
                                     'transition_number','heart rate_mean', 'mean blood pressure_mean',
       'oxygen saturation_mean', 'respiratory rate_mean',
       'systolic blood pressure_mean', 'hospital_expire_flag', 'gender_F',
       'gender_M', 'age_1', 'age_2', 'age_3', 'age_4',
       'ethnicity_AMERICAN INDIAN', 'ethnicity_ASIAN', 'ethnicity_BLACK',
       'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_WHITE',
       'first_careunit_CCU', 'first_careunit_CSRU', 'first_careunit_MICU',
       'first_careunit_SICU', 'first_careunit_TSICU'], axis=1)

In [41]:
X_merge_copy2.columns

Index(['Action', 'episode_id', 'episode_name', 'hospital_expire_flag',
       'transition_number', 'heart rate_mean', 'mean blood pressure_mean',
       'oxygen saturation_mean', 'respiratory rate_mean',
       'systolic blood pressure_mean', 'hospital_expire_flag', 'gender_F',
       'gender_M', 'age_1', 'age_2', 'age_3', 'age_4',
       'ethnicity_AMERICAN INDIAN', 'ethnicity_ASIAN', 'ethnicity_BLACK',
       'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_WHITE',
       'first_careunit_CCU', 'first_careunit_CSRU', 'first_careunit_MICU',
       'first_careunit_SICU', 'first_careunit_TSICU'],
      dtype='object')

In [42]:
header_list = ["action", "all_action_probabilities", "episode_id","episode_name", "reward", "transition_number",
"state_feature_0","state_feature_1","state_feature_2","state_feature_3","state_feature_4","state_feature_5",
"state_feature_6", "state_feature_7",
"state_feature_8",
"state_feature_9",
"state_feature_10",
"state_feature_11",
"state_feature_12",
"state_feature_13",
"state_feature_14",
"state_feature_15",
"state_feature_16",
"state_feature_17",
"state_feature_18",
"state_feature_19",
"state_feature_20",
"state_feature_21",
"state_feature_22"]



In [43]:
#X_merge_copy2 = X_merge_copy2.drop(['all_action_probabilities'],axis =1)

mylist_4 = [[0.04761904761]*21] * 2200954

X_merge_copy2.insert(1, "all_action_probabilities", mylist_4, True) 

X_merge_copy2.head()



Action  \
subject_id icustay_id hadm_id hours_in           
3          211552     145834  0             11   
                              1             17   
                              2             17   
                              3             16   
                              4             16   

                                                                 all_action_probabilities  \
subject_id icustay_id hadm_id hours_in                                                      
3          211552     145834  0         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              1         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              2         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              3         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              4         [0.04761904761, 0.04761904761, 0.04761904761, ...   

                                        episode_id episode_name  \
subject_id icustay_id hadm_id hours_in                            
3          211552     145834  0                  3     mimic_RL   
                              1                  3     mimic_RL   
                              2                  3     mimic_RL   
                              3                  3     mimic_RL   
                              4                  3     mimic_RL   

                                       hospital_expire_flag  \
subject_id icustay_id hadm_id hours_in                        
3          211552     145834  0                           0   
                              1                           0   
                              2                           0   
                              3                           0   
                              4                           0   

                                        transition_number  heart rate_mean  \
subject_id icustay_id hadm_id hours_in                                       
3          211552     145834  0                         0         0.476667   
                              1                         1         0.512500   
                              2                         2         0.459167   
                              3                         3         0.430833   
                              4                         4         0.486667   

                                        mean blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                             
3          211552     145834  0                         0.277075   
                              1                         0.137131   
                              2                         0.155767   
                              3                         0.223892   
                              4                         0.199367   

                                        oxygen saturation_mean  \
subject_id icustay_id hadm_id hours_in                           
3          211552     145834  0                         0.7400   
                              1                         0.7400   
                              2                         0.9850   
                              3                         0.9975   
                              4                         0.9800   

                                        respiratory rate_mean  ...  \
subject_id icustay_id hadm_id hours_in                         ...   
3          211552     145834  0                      0.053333  ...   
                              1                      0.051667  ...   
                              2                      0.023333  ...   
                              3                      0.017500  ...   
                              4                      0.045556  ...   

                                        ethnicity_ASIAN ethnicity_BLACK  \
subject_id icustay_id hadm_id hours_in                                  

In [45]:
X_merge_copy2.to_csv(r'~/documents/curated/mimic_rl_5action.csv',header = header_list, index = False)

### Generate no action list 

In [50]:
X_merge_copy2 = X_merge_copy2.drop(['Action'],axis =1)

mylist_5 = [0] * 2200954
X_merge_copy2.insert(0, "Action", mylist_5, True) 
X_merge_copy2.head()

Action  \
subject_id icustay_id hadm_id hours_in           
3          211552     145834  0              0   
                              1              0   
                              2              0   
                              3              0   
                              4              0   

                                                                 all_action_probabilities  \
subject_id icustay_id hadm_id hours_in                                                      
3          211552     145834  0         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              1         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              2         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              3         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              4         [0.04761904761, 0.04761904761, 0.04761904761, ...   

                                        episode_id episode_name  \
subject_id icustay_id hadm_id hours_in                            
3          211552     145834  0                  3     mimic_RL   
                              1                  3     mimic_RL   
                              2                  3     mimic_RL   
                              3                  3     mimic_RL   
                              4                  3     mimic_RL   

                                       hospital_expire_flag  \
subject_id icustay_id hadm_id hours_in                        
3          211552     145834  0                           0   
                              1                           0   
                              2                           0   
                              3                           0   
                              4                           0   

                                        transition_number  heart rate_mean  \
subject_id icustay_id hadm_id hours_in                                       
3          211552     145834  0                         0         0.476667   
                              1                         1         0.512500   
                              2                         2         0.459167   
                              3                         3         0.430833   
                              4                         4         0.486667   

                                        mean blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                             
3          211552     145834  0                         0.277075   
                              1                         0.137131   
                              2                         0.155767   
                              3                         0.223892   
                              4                         0.199367   

                                        oxygen saturation_mean  \
subject_id icustay_id hadm_id hours_in                           
3          211552     145834  0                         0.7400   
                              1                         0.7400   
                              2                         0.9850   
                              3                         0.9975   
                              4                         0.9800   

                                        respiratory rate_mean  ...  \
subject_id icustay_id hadm_id hours_in                         ...   
3          211552     145834  0                      0.053333  ...   
                              1                      0.051667  ...   
                              2                      0.023333  ...   
                              3                      0.017500  ...   
                              4                      0.045556  ...   

                                        ethnicity_ASIAN ethnicity_BLACK  \
subject_id icustay_id hadm_id hours_in                                  

In [51]:
X_merge_copy2.to_csv(r'~/documents/curated/mimic_rl_noaction.csv',header = header_list, index = False)

In [116]:
#santiy check
X_merge_copy2.iloc[400:430]

all_action_probabilities  \
subject_id icustay_id hadm_id hours_in                                                      
9          220597     150750  124       [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              125       [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              126       [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              127       [0.04761904761, 0.04761904761, 0.04761904761, ...   
11         229441     194540  0         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              1         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              2         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              3         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              4         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              5         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              6         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              7         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              8         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              9         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              10        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              11        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              12        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              13        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              14        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              15        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              16        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              17        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              18        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              19        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              20        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              21        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              22        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              23        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              24        [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              25        [0.04761904761, 0.04761904761, 0.04761904761, ...   

                                        Action  episode_id episode_name  \
subject_id icustay_id hadm_id hours_in                                    
9          220597     150750  124            0           9     mimic_RL   
                              125            0           9     mimic_RL   
                              126            0           9     mimic_RL   
                              127            0           9     mimic_RL   
11         229441     194540  0              0          11     mimic_RL   
                              1              0          11     mimic_RL   
                              2              0          11     mimic_RL   
                              3              0          11     mimic_RL   
                              4              0          11     mimic_RL   
                              5              0          11     mimic_RL   
                              6              0          11     mimic_RL   
                              7              0          11     mimic_RL   
                              8       

### Generate random actions list 

In [44]:
#randomlist = random.sample(range(0, 31), 2200954)

X_merge_copy2 = X_merge_copy2.drop(['Action'],axis =1)

randomlist = [random.randint(0,20) for iter in range(2200954)]

X_merge_copy2.insert(0, "Action", randomlist, True) 
X_merge_copy2.head()


Action  \
subject_id icustay_id hadm_id hours_in           
3          211552     145834  0              6   
                              1              0   
                              2             14   
                              3             17   
                              4              3   

                                                                 all_action_probabilities  \
subject_id icustay_id hadm_id hours_in                                                      
3          211552     145834  0         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              1         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              2         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              3         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              4         [0.04761904761, 0.04761904761, 0.04761904761, ...   

                                        episode_id episode_name  \
subject_id icustay_id hadm_id hours_in                            
3          211552     145834  0                  3     mimic_RL   
                              1                  3     mimic_RL   
                              2                  3     mimic_RL   
                              3                  3     mimic_RL   
                              4                  3     mimic_RL   

                                       hospital_expire_flag  \
subject_id icustay_id hadm_id hours_in                        
3          211552     145834  0                           0   
                              1                           0   
                              2                           0   
                              3                           0   
                              4                           0   

                                        transition_number  heart rate_mean  \
subject_id icustay_id hadm_id hours_in                                       
3          211552     145834  0                         0         0.476667   
                              1                         1         0.512500   
                              2                         2         0.459167   
                              3                         3         0.430833   
                              4                         4         0.486667   

                                        mean blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                             
3          211552     145834  0                         0.277075   
                              1                         0.137131   
                              2                         0.155767   
                              3                         0.223892   
                              4                         0.199367   

                                        oxygen saturation_mean  \
subject_id icustay_id hadm_id hours_in                           
3          211552     145834  0                         0.7400   
                              1                         0.7400   
                              2                         0.9850   
                              3                         0.9975   
                              4                         0.9800   

                                        respiratory rate_mean  ...  \
subject_id icustay_id hadm_id hours_in                         ...   
3          211552     145834  0                      0.053333  ...   
                              1                      0.051667  ...   
                              2                      0.023333  ...   
                              3                      0.017500  ...   
                              4                      0.045556  ...   

                                        ethnicity_ASIAN ethnicity_BLACK  \
subject_id icustay_id hadm_id hours_in                                  

In [45]:
unique(randomlist)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [46]:
X_merge_copy2.to_csv(r'~/documents/curated/mimic_rl_rand_action.csv',header = header_list, index = False)

In [48]:
X_merge_copy2.head()

all_action_probabilities  \
subject_id icustay_id hadm_id hours_in                                                      
3          211552     145834  0         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              1         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              2         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              3         [0.04761904761, 0.04761904761, 0.04761904761, ...   
                              4         [0.04761904761, 0.04761904761, 0.04761904761, ...   

                                        Action  episode_id episode_name  \
subject_id icustay_id hadm_id hours_in                                    
3          211552     145834  0              1           3     mimic_RL   
                              1             21           3     mimic_RL   
                              2             12           3     mimic_RL   
                              3              0           3     mimic_RL   
                              4             14           3     mimic_RL   

                                       hospital_expire_flag  \
subject_id icustay_id hadm_id hours_in                        
3          211552     145834  0                           0   
                              1                           0   
                              2                           0   
                              3                           0   
                              4                           0   

                                        transition_number  heart rate_mean  \
subject_id icustay_id hadm_id hours_in                                       
3          211552     145834  0                         0         0.476667   
                              1                         1         0.512500   
                              2                         2         0.459167   
                              3                         3         0.430833   
                              4                         4         0.486667   

                                        mean blood pressure_mean  \
subject_id icustay_id hadm_id hours_in                             
3          211552     145834  0                         0.277075   
                              1                         0.137131   
                              2                         0.155767   
                              3                         0.223892   
                              4                         0.199367   

                                        oxygen saturation_mean  \
subject_id icustay_id hadm_id hours_in                           
3          211552     145834  0                         0.7400   
                              1                         0.7400   
                              2                         0.9850   
                              3                         0.9975   
                              4                         0.9800   

                                        respiratory rate_mean  ...  \
subject_id icustay_id hadm_id hours_in                         ...   
3          211552     145834  0                      0.053333  ...   
                              1                      0.051667  ...   
                              2                      0.023333  ...   
                              3                      0.017500  ...   
                              4                      0.045556  ...   

                                        ethnicity_ASIAN ethnicity_BLACK  \
subject_id icustay_id hadm_id hours_in                                    
3          211552     145834  0                       0               0   
                              1                       0               0   
                              2                       0               0   
                              3                       0               0   
                 